In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import pickle
model = pickle.load(open('classification_model.sav', 'rb'))

In [2]:
df = pd.read_csv('AW_Test_prepped_classification.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
CustomerID           500 non-null int64
CountryRegionName    500 non-null object
Education            500 non-null object
Occupation           500 non-null object
Gender               500 non-null object
MaritalStatus        500 non-null object
NumberCarsOwned      500 non-null int64
TotalChildren        500 non-null int64
AgeRange             500 non-null object
log_YearlyIncome     500 non-null float64
dtypes: float64(1), int64(3), object(6)
memory usage: 39.1+ KB


In [3]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

In [4]:
categorical_columns = ['Education', 'Occupation', 'Gender', 'MaritalStatus', 'AgeRange']
Features = encode_string(df['CountryRegionName'])
for col in categorical_columns:
    temp = encode_string(df[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])

(500, 24)
[[0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.]]


In [5]:
Features = np.concatenate([Features, np.array(df[['NumberCarsOwned', 
                            'TotalChildren', 'log_YearlyIncome']])], axis = 1)
print(Features.shape)
print(Features[:2, :]) 

(500, 27)
[[ 0.          0.          0.          0.          0.          1.
   1.          0.          0.          0.          0.          0.
   1.          0.          0.          0.          1.          0.
   0.          1.          0.          1.          0.          0.
   2.          5.         11.37286998]
 [ 0.          1.          0.          0.          0.          0.
   1.          0.          0.          0.          0.          0.
   0.          0.          0.          1.          0.          1.
   1.          0.          1.          0.          0.          0.
   2.          4.         11.51417468]]


In [6]:
scaler = preprocessing.StandardScaler().fit(Features[:,25:])
Features[:,25:] = scaler.transform(Features[:,25:])

In [18]:
ClassPred = pd.DataFrame(model.predict(Features))
ClassPred.rename(columns = {0: 'PredBikeBuyer'}, inplace=True)
ClassPred['CustomerID'] = df['CustomerID']
ClassPred = ClassPred[['CustomerID', 'PredBikeBuyer']]
ClassPred.to_csv('ClassPred.csv', mode='w', index=False)